# Failure Modes

In [1]:
from rubicon_ml import Rubicon


rb = Rubicon(persistence="memory")
rb.get_project(name="failure modes")

RubiconException: No project with name 'failure modes' found.

In [2]:
from rubicon_ml.client.utils.exception_handling import set_failure_mode


set_failure_mode("warn")

rb.get_project(name="failure modes")

/Users/nvd215/github/capitalone/rubicon-ml/rubicon_ml/client/utils/exception_handling.py:25: UserWarning: RubiconException("No project with name 'failure modes' found.")
  warnings.warn(repr(e))


In [3]:
set_failure_mode("log")

rb.get_project(name="failure modes")

ERROR:root:RubiconException("No project with name 'failure modes' found.")


text about setting back to "raise" for default behavior

### aside on return values

In [4]:
rb.create_project(name="failure modes")
project = rb.get_project(name="failure modes")

print(project)

Project(name='failure modes', id='1f71926a-8445-48bb-90b0-9a8a47fd2321', description=None, github_url=None, training_metadata=None, created_at=datetime.datetime(2022, 11, 15, 0, 41, 22, 738553))


In [5]:
set_failure_mode("log")

project = rb.get_project(name="failure modes v2")
print(project)

ERROR:root:RubiconException("No project with name 'failure modes v2' found.")


None


### more practical use

In [6]:
from sklearn.neighbors import KNeighborsClassifier


class BrokenFilesystem:
    pass

set_failure_mode("raise")

In [7]:
project = rb.create_project(name="failure modes v3")
experiment = project.log_experiment()

experiment

In [8]:
rb.config.repository.filesystem = BrokenFilesystem()

In [9]:
knn = KNeighborsClassifier(n_neighbors=1)
X_train, y_train, X_test, y_test = [[0, 1, 2, 3]], [0], [[0, 1, 2, 3]], [0]

knn.fit(X_train, y_train)
experiment.log_parameter(name="n_neighbors", value=1)

score = knn.score(X_test, y_test)
experiment.log_metric(name="score", value=score)

score

AttributeError: 'BrokenFilesystem' object has no attribute 'exists'

In [10]:
set_failure_mode("log")

knn = KNeighborsClassifier(n_neighbors=1)
X_train, y_train, X_test, y_test = [[0, 1, 2, 3]], [0], [[0, 1, 2, 3]], [0]

knn.fit(X_train, y_train)
experiment.log_parameter(name="n_neighbors", value=1)

score = knn.score(X_test, y_test)
experiment.log_metric(name="score", value=score)

score

ERROR:root:AttributeError("'BrokenFilesystem' object has no attribute 'exists'")
ERROR:root:AttributeError("'BrokenFilesystem' object has no attribute 'exists'")


1.0